<a href="https://colab.research.google.com/github/sahildawar/Car-Speed-Measure/blob/main/Car_Speed_Measure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio opencv-python
import cv2
import torch
import numpy as np
from collections import defaultdict
from google.colab import files

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

In [ ]:
video_path = '/content/Untitled video - Made with Clipchamp.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:
out = cv2.VideoWriter('output_with_speed.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps,
                      (width, height))

In [ ]:
car_tracks = defaultdict(list)
prev_centroids = {}
next_id = 0
frame_no = 0
pixel_to_meter = 0.05

In [ ]:
def assign_id(new_centroid, prev_centroids, threshold=50):
    for car_id, prev_centroid in prev_centroids.items():
        dist = np.linalg.norm(np.array(new_centroid) - np.array(prev_centroid))
        if dist < threshold:
            return car_id
    return None


In [ ]:
from ultralytics.utils import ops  # Make sure this is imported

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]  # YOLOv8 returns a list; get the first result

    boxes = results.boxes.xyxy.cpu().numpy()        # (xmin, ymin, xmax, ymax)
    confidences = results.boxes.conf.cpu().numpy()  # confidence scores
    class_ids = results.boxes.cls.cpu().numpy()     # class indices
    current_centroids = {}

    for i, box in enumerate(boxes):
        class_id = int(class_ids[i])
        class_name = model.names[class_id]

        if class_name != 'car':
            continue

        xmin, ymin, xmax, ymax = map(int, box)
        center_x = int((xmin + xmax) / 2)
        center_y = int((ymin + ymax) / 2)
        centroid = (center_x, center_y)

        car_id = assign_id(centroid, prev_centroids)
        if car_id is None:
            car_id = next_id
            next_id += 1

        car_tracks[car_id].append((center_x, center_y, frame_no))
        current_centroids[car_id] = centroid

        speed_text = f"ID:{car_id} | Speed: -- km/h"
        if len(car_tracks[car_id]) >= 2:
            x1, y1, f1 = car_tracks[car_id][-2]
            x2, y2, f2 = car_tracks[car_id][-1]
            pixel_dist = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            meter_dist = pixel_dist * pixel_to_meter
            time_elapsed = (f2 - f1) / fps
            if time_elapsed > 0:
                speed = (meter_dist / time_elapsed) * 3.6  # Convert to km/h
                speed_text = f"ID:{car_id} | {speed:.1f} km/h"

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(frame, speed_text, (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    prev_centroids = current_centroids.copy()
    frame_no += 1
    out.write(frame)

cap.release()
out.release()
files.download('output_with_speed.avi')


In [ ]:
from google.colab import files
files.download('output_with_speed.avi')
